In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from datetime import datetime
import datetime as dt
import time

#Read the data - SAP, SFDC & Ecomm
SAP = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\NAC Report\\01_SAP_Raw Report_NAC.xlsx")
SFDC = pd.read_csv("C:\\WSS-NAC-Reports\\Automation Folder\\NAC Report\\SFDC_Raw Nac Report.csv", parse_dates=['Date/Time Opened'], dayfirst=True, encoding = "cp1252")

#Remove duplicates on both data
SAP = SAP.drop_duplicates(subset=['Sales Document'])
SFDC = SFDC.drop_duplicates(subset=['SAP Order No.'])

#Remove false time from Document Date, Delivery Date & Goods Issue Date
SAP['Goods Issue Date'] = SAP['Goods Issue Date'].dt.strftime('%d/%m/%Y')
SAP['Document Date'] = SAP['Document Date'].dt.strftime('%d/%m/%Y')
SAP['Delivery Date'] = SAP['Delivery Date'].dt.strftime('%d/%m/%Y')

#Adjust SFDC date/time
SFDC['Date/Time Opened'] = (SFDC['Date/Time Opened']).dt.strftime('%Y-%m-%d %H:%M:%S')

SFDC.rename(columns = {'Case Number' : 'Case_Number', 'Date/Time Opened' : 'Rcvd_SFDC', 
                       'Number of subscribers activated' : 'Number_of_Subs_activated', 
                       'Number of subscribers renewed' : 'Number_of_Subs_Renewed'}, inplace = True)
SAP.rename(columns = {'Sales Office' : 'Order_Type', 'Goods Issue Date' : 'Goods_Date'}, inplace = True)



In [2]:
#Only show Sales Office from 9503 & remove sales group 930

values = [9503]
values2 = [930]

SAP = SAP.loc[SAP['Order_Type'].isin(values)]
SAP = SAP.loc[~SAP['Sales Group'].isin(values2)]

#Merge both data

SFDC['SAP Order No.']=SFDC['SAP Order No.'].apply(pd.to_numeric, errors='coerce').fillna(0).astype(float).dropna()
C1 = pd.merge(left=SAP, right=SFDC, left_on='Sales Document', right_on='SAP Order No.', how='left')

#Read the agents' list

agt = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\NAC Report\\Agent list.xlsx")

#Merge agent list data on C1

C2 = pd.merge(left=C1, right=agt, left_on='User ID', right_on='ID', how='left')

#Remove non NAC orders

C2.dropna(subset=['Comment'], inplace=True)

#Define all the necessary status

p2 = pd.read_csv("C:\\WSS-NAC-Reports\\Automation Folder\\NAC Report\\SFDC_Raw Nac Report.csv", parse_dates=['Edit Date'], dayfirst=True, encoding = "cp1252")

#Adjust date/time
p2['Edit Date'] = (p2['Edit Date']).dt.strftime('%Y-%m-%d %H:%M:%S')

p2['ACT_Status_Date'] = p2['Edit Date']
p2['CARAT Date'] = p2['Edit Date']
p2['CAD Date'] = p2['Edit Date']
p2['CARAT'] = p2['Case Number']
p2['VALIDATION'] = p2['Case Number']
p2['PREORDER'] = p2['Case Number']
p2['BACKORDER'] = p2['Case Number']
p2['CAD'] = p2['Case Number']

#Check for Activation

act2 = p2[(p2['Old Value'] == 'Activation') | (p2['New Value'] == 'Activation')]
act2 = act2[['Case Number', 'ACT_Status_Date']]
act2 = act2.drop_duplicates(subset=['Case Number'])

#Check for Backorder

bo2 = p2[(p2['Old Value'] == 'Backorder') | (p2['New Value'] == 'Backorder') | 
         (p2['Old Value'] == 'Allocation') | (p2['New Value'] == 'Allocation')]
bo2 = bo2[['Case Number', 'BACKORDER']]
bo2 = bo2.drop_duplicates(subset=['Case Number'])

#Check for CARAT

car2 = p2[(p2['Old Value'] == 'CARAT') | (p2['New Value'] == 'CARAT')]
car2 = car2[['Case Number', 'CARAT Date', 'CARAT']]
car2 = car2.drop_duplicates(subset=['Case Number'])

#Check for Validation

val2 = p2[(p2['Old Value'] == 'Validation') | (p2['New Value'] == 'Validation')]
val2 = val2[['Case Number', 'VALIDATION']]
val2 = val2.drop_duplicates(subset=['Case Number'])

#Check for Preorder

pre2 = p2[(p2['Old Value'] == 'Preorder') | (p2['New Value'] == 'Preorder')]
pre2 = pre2[['Case Number', 'PREORDER']]
pre2 = pre2.drop_duplicates(subset=['Case Number'])

#Check for CAD

cad2 = p2[(p2['Old Value'] == 'CAD') | (p2['New Value'] == 'CAD')]
cad2 = cad2[['Case Number', 'CAD Date', 'CAD']]
cad2 = cad2.drop_duplicates(subset=['Case Number'])

#Merge all status

from functools import reduce

data_frame = [act2, car2, bo2, val2, pre2, cad2]
status2 = reduce(lambda left, right: pd.merge(left, right, on=['Case Number'], how='outer'), data_frame)

f = status2[['CARAT', 'BACKORDER', 'VALIDATION', 'PREORDER']]
status2['ALL_EXCL'] = [ x[pd.notna(x)].tolist()[0] if len(x[pd.notna(x)].tolist()) > 0 else "" for x in f.values]

#Add the various status to table

C3 = pd.merge(left=C2, right=status2, left_on='Case_Number', right_on='Case Number', how='left')
#Read SAP again so as to concatenate date & time

sp = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\NAC Report\\01_SAP_Raw Report_NAC.xlsx")
sp['Created_in_SAP'] = pd.to_datetime(sp['Document Date']) + pd.to_timedelta(sp['Document Time'].astype(str))
sp['Dropped_to_WH'] = pd.to_datetime(sp['Delivery Date']) + pd.to_timedelta(sp['Delivery Time'].astype(str))
sp['Goods_Issue'] = pd.to_datetime(sp['Goods Issue Date']) + pd.to_timedelta(sp['Goods Issue Time'].astype(str))
sp.rename(columns = {'Sales Document' : 'Sales Doc'}, inplace = True)
sp_new = sp[['Sales Doc', 'Created_in_SAP', 'Dropped_to_WH', 'Goods_Issue']]

#Remove duplicate on sales document
sp_new = sp_new.drop_duplicates(subset=['Sales Doc'])


#Add SFDC recieved time to the sp table by joining sp to C3 using sale document column

C4 = pd.merge(left=C3, right=sp_new, left_on='Sales Document', right_on='Sales Doc', how='left')



In [3]:
#no_of_lines

C4['no_of_Lines'] = C4['Number_of_Subs_activated'] + C4['Number_of_Subs_Renewed']


In [4]:
#Calculate date without weekends and holidays

COL11 = "SFDC_Goods_Mins"
COL12 = "SFDC_SAP_Mins"
COL13 = "SAP_WH_Mins"
COL14 = "WH_Shipping_Mins"
COL15 = "SAP_Goods_Mins"

hd = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\IQ Report\\Holidays.xlsx")
HOLIDAYS = hd['HolidayDate']

def get_diff(date1, date2):
    try:
        diff = abs((date2 - date1).total_seconds())
        days_between = (date2.date() - date1.date()).days
        weekdays_between = abs(np.busday_count(date2.date(), date1.date()))
        weekends_between = days_between - weekdays_between

        if date2.date() != date1.date():
            diff = diff - (weekends_between * SECONDS_IN_A_DAY)
            
        for holiday in HOLIDAYS:
            holiday_date = datetime.strptime(str(holiday), "%Y-%m-%d %H:%M:%S")
            if date1.date() <= holiday_date.date() <= date2.date():
                diff = diff -  SECONDS_IN_A_DAY


        return int(diff / 60)

    except Exception as e:
        print(e)
        return 0


SECONDS_IN_A_DAY = 86400
C4[COL11] = 0
C4[COL12] = 0
C4[COL13] = 0
C4[COL14] = 0
C4[COL15] = 0

for index, row in C4.iterrows():
    try:
        sfdc_date = datetime.strptime(str(row["Rcvd_SFDC"]), "%Y-%m-%d %H:%M:%S")
        doc_time_date = datetime.strptime(str(row["Created_in_SAP"]), "%Y-%m-%d %H:%M:%S")
        del_time_date = datetime.strptime(str(row["Dropped_to_WH"]), "%Y-%m-%d %H:%M:%S")
        gd_time_date = datetime.strptime(str(row["Goods_Issue"]), "%Y-%m-%d %H:%M:%S")
           
        C4.at[index, COL11] = get_diff(sfdc_date, gd_time_date)
        C4.at[index, COL12] = ifor_val = get_diff(sfdc_date, doc_time_date)
        C4.at[index, COL13] = get_diff(doc_time_date, del_time_date)
        C4.at[index, COL14] = get_diff(del_time_date, gd_time_date)
        C4.at[index, COL15] = get_diff(doc_time_date, gd_time_date)
    except Exception as e:
        print(e)
        


In [5]:
COL16 = "WH_ACT_Mins"
COL17 = "ACT_Shipping_Mins"

hd = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\IQ Report\\Holidays.xlsx")
HOLIDAYS = hd['HolidayDate']

def get_diff(date1, date2):
    try:
        diff = abs((date2 - date1).total_seconds())
        days_between = (date2.date() - date1.date()).days
        weekdays_between = abs(np.busday_count(date2.date(), date1.date()))
        weekends_between = days_between - weekdays_between

        if date2.date() != date1.date():
            diff = diff - (weekends_between * SECONDS_IN_A_DAY)
            
        for holiday in HOLIDAYS:
            holiday_date = datetime.strptime(str(holiday), "%Y-%m-%d %H:%M:%S")
            if date1.date() <= holiday_date.date() <= date2.date():
                diff = diff -  SECONDS_IN_A_DAY


        return int(diff / 60)

    except Exception as e:
        print(e)
        return 0


SECONDS_IN_A_DAY = 86400
C4[COL16] = 0
C4[COL17] = 0

for index, row in C4.iterrows():
    try:
        act_time_date = datetime.strptime(str(row["ACT_Status_Date"]), "%Y-%m-%d %H:%M:%S")
        del_time_date = datetime.strptime(str(row["Dropped_to_WH"]), "%Y-%m-%d %H:%M:%S")
        gd_time_date = datetime.strptime(str(row["Goods_Issue"]), "%Y-%m-%d %H:%M:%S")
        
        C4.at[index, COL16] = get_diff(del_time_date, act_time_date)
        C4.at[index, COL17] = get_diff(act_time_date, gd_time_date)
        
    except Exception as e:
        print(e)

In [6]:
COL18 = "CARAT_Processtime_Mins"

hd = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\IQ Report\\Holidays.xlsx")
HOLIDAYS = hd['HolidayDate']

def get_diff(date1, date2):
    try:
        diff = abs((date2 - date1).total_seconds())
        days_between = (date2.date() - date1.date()).days
        weekdays_between = abs(np.busday_count(date2.date(), date1.date()))
        weekends_between = days_between - weekdays_between

        if date2.date() != date1.date():
            diff = diff - (weekends_between * SECONDS_IN_A_DAY)
            
        for holiday in HOLIDAYS:
            holiday_date = datetime.strptime(str(holiday), "%Y-%m-%d %H:%M:%S")
            if date1.date() <= holiday_date.date() <= date2.date():
                diff = diff -  SECONDS_IN_A_DAY


        return int(diff / 60)

    except Exception as e:
        print(e)
        return 0


SECONDS_IN_A_DAY = 86400
C4[COL18] = 0

for index, row in C4.iterrows():
    try:
        car_time_date = datetime.strptime(str(row["CARAT Date"]), "%Y-%m-%d %H:%M:%S")
        cad_time_date = datetime.strptime(str(row["CAD Date"]), "%Y-%m-%d %H:%M:%S")
        
        C4.at[index, COL18] = get_diff(car_time_date, cad_time_date)
    except Exception as e:
        print(e)

time data 'nan' does not match format '%Y-%m-%d %H:%M:%S'


In [7]:
#For CBAN cad
C5 = C4.dropna(subset = ['CAD'])


In [8]:
#Export CBAN_Cad Result

CBAN = C5[['Sales Document', 'Case_Number', 'Name of sold-to party', 'User ID', 'Agent_Name',  'Rcvd_SFDC','Created_in_SAP', 
         'Dropped_to_WH', 'ACT_Status_Date', 'Goods_Issue', 'Goods_Date', 'SFDC_Goods_Mins', 'SFDC_SAP_Mins', 'SAP_WH_Mins', 'WH_ACT_Mins', 
         'ACT_Shipping_Mins', 'WH_Shipping_Mins', 'SAP_Goods_Mins', 'Number_of_Subs_activated', 'Number_of_Subs_Renewed', 
          'CARAT_Processtime_Mins', 'Order_Type', 'Sales Group', 'BACKORDER', 'VALIDATION', 'PREORDER', 'CARAT', 'ALL_EXCL', 'no_of_Lines']]
export_excell = CBAN.to_excel(r'C:\\WSS-NAC-Reports\\Automation Folder\\NAC Report\\CBAN_CAD.xlsx', index=None, header=True)        

In [9]:
#For CBAN no cad

C6 = C4[C4['CAD'].isnull()]


In [10]:
#Export CBAN_No-Cad Result

CBAN = C6[['Sales Document', 'Case_Number', 'Name of sold-to party', 'User ID', 'Agent_Name',  'Rcvd_SFDC','Created_in_SAP', 
         'Dropped_to_WH', 'ACT_Status_Date', 'Goods_Issue', 'Goods_Date', 'SFDC_Goods_Mins', 'SFDC_SAP_Mins', 'SAP_WH_Mins', 'WH_ACT_Mins', 
         'ACT_Shipping_Mins', 'WH_Shipping_Mins', 'SAP_Goods_Mins', 'Number_of_Subs_activated', 'Number_of_Subs_Renewed', 
          'CARAT_Processtime_Mins', 'Order_Type', 'Sales Group', 'BACKORDER', 'VALIDATION', 'PREORDER', 'CARAT', 'ALL_EXCL', 'no_of_Lines']]
export_excell = CBAN.to_excel(r'C:\\WSS-NAC-Reports\\Automation Folder\\NAC Report\\CBAN_NOCAD.xlsx', index=None, header=True)        